In [3]:
#q8
!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 25821052928 (24.05 GB)
Present Capacity: 19291622229 (17.97 GB)
DFS Remaining: 19028799488 (17.72 GB)
DFS Used: 262822741 (250.65 MB)
DFS Used%: 1.36%
Replicated Blocks:
	Under replicated blocks: 167
	Blocks with corrupt replicas: 0
	Missing blocks: 85
	Missing blocks (with replication factor 1): 85
	Low redundancy blocks with highest priority to recover: 167
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 172.19.0.3:9866 (project-4-pizza-dn-1.project-4-pizza_default)
Hostname: 39d01ba5ab91
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 262822741 (250.65 MB)
Non DFS Used: 6512653483 (6.07 GB)
DFS Remaining: 19028799488 (17.72 GB)
DFS Used%: 1.02%
DFS Remaini

In [3]:
import time
time.sleep(30)

KeyboardInterrupt: 

In [6]:
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=1 -cp hdma-wi-2021.csv hdfs://boss:9000/single.csv

cp: `hdfs://boss:9000/single.csv': File exists


In [8]:
#q9
import requests
import pyarrow as pa
import pyarrow.fs
url = "http://boss:9870/webhdfs/v1/single.csv?op=OPEN"
container_number_count = dict()


for offset in range(0, 174944099, 1048576):
    url_with_offset = f"{url}&offset={offset}"
    r = requests.get(url_with_offset,stream = True)
    if r.status_code == 403:
        if "lost" not in container_number_count:
            container_number_count['lost'] = 1
        else:
            container_number_count['lost'] +=1
    if r.status_code == 200:
        url_array = r.url.split('/')
        # print(url_array)
        container_id = url_array[2]
        container = container_id.split(':')
        final_container_id = container[0]
        if final_container_id not in container_number_count:
            container_number_count[final_container_id] = 1
        else:
            container_number_count[final_container_id] += 1

print(dict(container_number_count))

{'lost': 85, '39d01ba5ab91': 82}


In [39]:
#q10


count = 0
good_blocks = []
url = "http://boss:9870/webhdfs/v1/single.csv?op=OPEN"
for offset in range(0, 174944099, 1048576):
    url_with_offset = f"{url}&offset={offset}"
    r = requests.get(url_with_offset,stream = True)
    if r.status_code == 403:
        continue
    if r.status_code == 200:
        url_array = r.url
        good_blocks.append(url_array.split("=")[-1])


for i in good_blocks:
    hdfs = pa.fs.HadoopFileSystem("boss", 9000)
    with hdfs.open_input_file('hdfs://boss:9000/single.csv') as f:
        reader = f.read_at(1048576,int(i))
        count += str(reader).count("Single Family")
count

217970